In [ ]:
from pathlib import Path

seeds = range(1, 11)

# Config should match Problem init

base_path = Path("smacbenchmarking/configs/problem")

# Problem ids
benchmark_id_bbob = "BBOB"

# BBOB

In [ ]:
# BBOB
from omegaconf import DictConfig, OmegaConf

import shutil

target_path = base_path / benchmark_id_bbob
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "smacbenchmarking.benchmarks.bbob.BBOBProblem"

fids = list(range(1, 25))
instances = [0]
dimensions = [4]
n_trials = {
    4: 100,
}

for fid in fids:
    for inst in instances:
        for dim in dimensions:
            problem_id = f"noiseless/{dim}/{fid}/{inst}"
            cfg = DictConfig({
                "benchmark_id": benchmark_id_bbob,
                "problem_id": problem_id,
                "problem": {
                    # "problem_class": {
                        "_target_": problem_class,
                        "dimension": dim,
                        "fid": fid,
                        "instance": inst,
                        "seed": "${seed}",  # get the seed from global cfg                      
                    # },
                },
                "task": {
                    "n_trials": n_trials[dim],
                    "n_objectives": 1,
                }
            
            })
            fn = target_path / f"cfg_{dim}_{fid}_{dim}_{inst}.yaml"
            yaml_str = OmegaConf.to_yaml(cfg=cfg)
            yaml_str = "# @package _global_\n" + yaml_str
            fn.write_text(yaml_str)
            print(cfg)


# HPOB

In [ ]:
# HPOB
from operator import attrgetter
import re
from pathlib import Path

base_path = Path("smacbenchmarking/configs/problem")

# Problem ids
benchmark_id = "HPOB"

surrogates_dir = Path('smacbenchmarking/benchmark_data/HPO-B/saved-surrogates')

all_res = map(attrgetter('name'), surrogates_dir.glob('surrogate-*'))

from omegaconf import DictConfig, OmegaConf
import shutil

target_path = base_path / benchmark_id
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "smacbenchmarking.benchmarks.hpo_b.HPOBProblem"
n_trials = 100

for res in all_res:
    model_id, dataset_id = re.findall('\d+', res)
    problem_id = f'hpob/{model_id}/{dataset_id}'
    cfg = DictConfig({
        "benchmark_id": benchmark_id,
        "problem_id": problem_id,
        "problem": {
            # "problem_class": {
            "_target_": problem_class,
            "model_id": model_id,
            "dataset_id": dataset_id,
            "surrogates_dir": str(surrogates_dir)
            # },
        },
        "task": {
            "n_trials": n_trials,
            "n_objectives": 1,
        }

    })
    fn = target_path / f"cfg_{model_id}_{dataset_id}.yaml"
    yaml_str = OmegaConf.to_yaml(cfg=cfg)
    yaml_str = "# @package _global_\n" + yaml_str
    fn.write_text(yaml_str)


# HPOBench

In [ ]:
# Problem id
benchmark_id_hpobench = "HPOBench"

# HPOBench
from omegaconf import DictConfig, OmegaConf

import shutil

target_path = base_path / benchmark_id_hpobench
if target_path.is_dir():
    shutil.rmtree(str(target_path))
target_path.mkdir(parents=True, exist_ok=True)

problem_class = "smacbenchmarking.benchmarks.hpo_bench.HPOBenchProblem"

MODELS = ["lr", "nn", "rf", "svm", "xgboost"]

TASK_IDS = [10101, 53, 146818, 146821, 9952, 146822, 31, 3917]
TASK_IDS_LARGE = [168912, 3, 167119, 12, 146212, 168911, 9981, 167120, 14965, 146606, 7592, 9977]

# Choices according to https://arxiv.org/pdf/2109.06716.pdf, page 22
UPPER = {'subsample': 1.0,
         'iter':
             {
                 'lr': 1000,
                 'nn': 243,
         },
         'n_estimators': {
             'xgboost': 2000,
             'rf': 512,
         }}
LOWER = {'subsample': 0.1,
         'iter':
             {
                 'lr': 10,
                 'nn': 3,
         },
         'n_estimators': {
             'xgboost': 50,
             'rf': 16,
         }}

FIDELITY_TYPES = {'lr': ['iter', 'subsample'],
                  'nn': ['iter', 'subsample'],
                  'rf': ['n_estimators', 'subsample'],
                  'svm': ['subsample'],
                  'xgboost': ['n_estimators', 'subsample']}

n_trials = 100

for model in MODELS:
    # task_ids_large should be used for all models except the neural network
    tids = TASK_IDS if model=="nn" else TASK_IDS + TASK_IDS_LARGE
    for tid in tids:
        for fid_type in FIDELITY_TYPES[model]:
            
            if fid_type == 'n_estimators':
                lower = LOWER['n_estimators'][model]
                upper = UPPER['n_estimators'][model]
            elif fid_type == 'iter':
                lower = LOWER['iter'][model]
                upper = UPPER['iter'][model]
            else:
                lower = LOWER[fid_type]
                upper = UPPER[fid_type]
            
            problem_id = f"ml/{model}/{tid}"
            
            cfg = DictConfig({
                "benchmark_id": benchmark_id_hpobench,
                "problem_id": problem_id,
                "problem": {
                        "_target_": problem_class,
                        "model": model,
                        "task_id": tid,
                        "budget_type": fid_type,
                        "seed": "${seed}",  # get the seed from global cfg                      
                },
                "task": {
                    "n_trials": n_trials,
                    "n_objectives": 1,
                    'min_budget': lower,
                    'max_budget': upper,
                }

            })
            fn = target_path / f"cfg_ml_{model}_{tid}_{fid_type}.yaml"
            yaml_str = OmegaConf.to_yaml(cfg=cfg)
            yaml_str = "# @package _global_\n" + yaml_str
            fn.write_text(yaml_str)
            print(cfg)